# Caption Analysis

In [8]:
import tiktoken
import pandas as pd
import openai
from time import sleep
import os
file_path = "../../openai_key.txt"

with open(file_path, "r") as file:
    key = file.read()
openai.api_key = key
import numpy as np
import json
import time

In [9]:
def get_questions_anwers(prompt):
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user",
         "content": prompt}
    ])
    questions_answers = completion.choices[0].message.content
    return questions_answers

In [10]:
def caption_processing(caption):
    remove_words =["in this image i can see ", "in this image we can see ",  "this is an image of ", "a cartoon of ", "simpson ", "homer ", "a cartoon of ", "cartoon ", "this is an image of "]
    for remove_word in remove_words:
        caption = caption.replace(remove_word, "")
    replaced_by_person_words = ["duck", "bear", " character"]
    for word in replaced_by_person_words:
        caption.replace(word, "person")
    caption.replace("characters", "people")
    caption.replace("simpsons", "people")
    return caption + ". people skin is yellow."

In [11]:
def get_img_pth(img):
    ss = img[:3]
    image_path = ss + "/" + img + ".jpg"
    return image_path

In [12]:
def extract_cases(string):
    string = string.replace("For case", "Case")
    string = string.replace("For Case", "Case")
    string = string.replace(":\n", ":")
    
    cases = []
    
    # Split the string by 'Case' to separate each case
    case_strings = string.split('Case ')[1:]
    
    for case_string in case_strings:
        case = {}
        
        # Split the case string by newline character '\n' to separate each question-answer pair
        pairs = case_string.split('\nQ:')[1:]
        
        case['questions'] = []
        case['answers'] = []
        case['topics'] = []
        
        for pair in pairs:
            # Split the pair by 'Q:', 'A:', and 'Topic:' to extract the question, answer, and topic respectively
            question, answer, topic = pair.split(' A:', 1)[0].strip(), pair.split(' A:', 1)[1].split(' Topic:', 1)[0].strip(), pair.split(' Topic:', 1)[1].strip()
            
            case['questions'].append(question)
            case['answers'].append(answer)
            case['topics'].append(topic)
        
        cases.append(case)
    
    return cases

In [14]:
def chatgpt(sub_df, idx, requirement, start_time):
    caption_1 = list(sub_df["caption_1"])
    caption_2 = list(sub_df["caption_2"])
    prompt = requirement
    for i, (c1,c2) in enumerate(zip(caption_1, caption_2)):
        prompt += f"\nCase {i+1}\n"
        prompt += f"Description 1: {c1}\n"
        prompt += f"Description 2: {c2}\n"
    success = False
    while not success:
        try:
            string_result = get_questions_anwers(prompt)
            success = True
        except:
            print("retrying")
            sleep(5)
    if (idx + 1) % 2 == 0:
        print("-"*50)
        print(f"Processing {((idx+1)*3)} captions")
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"running time: {round(elapsed_time, 0)} seconds")
    cases = extract_cases(string_result)
    
    return cases

In [49]:
requirement = "You are given three cases with two different descriptions of an image each. " + \
    "You are required to ask 10 questions, their topics, and their answers based on the two descriptions for each case. " + \
    "The questions must be suitable for preschool students and should be about counting, reasoning, object detection, " + \
    "action recognition, attribute classification, comparative, spatial, or position. " + \
    "Morevoer, the type of question should be yes/no and open-end and the answer cannot be more than 3 word." + \
    "\ncase 0:\n" + \
    "Description 1: a man is standing in front of a large screen that is turned on he is wearing a blue shirt andblue headphones he is also wearing a purple shirt and a purple vest there is a large mirror on the wall behind the man.\n" + \
    "Description 2: a person is standing and he is wearing headphones on his head he is holding a microphone in his handhe is also wearing a blue tshirt and i can also see a microphone on the right side of the image there is a wall.\n" + \
    "case 0:\n" + \
    "Q:How many people are there? A:1 Topic:Counting.\n" + \
    "Q:What is the man doing? A:Standing Topic:Action recognition.\n" + \
    "Q:Is there a microphone? A:Yes Topic:Object detection.\n" + \
    "Q:How many microphones? A:1 Topic:Counting.\n" + \
    "Q:What color is the man's shirt? A:Blue Topic:Attribute classification.\n" + \
    "Q:What is hanging on the wall? A:Mirror Topic:Object detection.\n" + \
    "Q:What is he standing in front of? A:Screen Topic:Object detection.\n" + \
    "Q:Is he taller than the mirror? A:Unknown Topic:Comparative.\n" + \
    "Q:Where is the mirror hanging? A:Wall Topic:Position.\n" + \
    "Q:What is the shape of the mirror? A:Unknown Topic:Attribute classification.\n" + \
    "Q:How in the man feelining? A:Unknown Topic:Reasoning.\n"

In [69]:
print(requirement)

You are given three cases with two different descriptions of an image each. You are required to ask 10 questions, their topics, and their answers based on the two descriptions for each case. The questions must be suitable for preschool students and should be about counting, reasoning, object detection, action recognition, attribute classification, comparative, spatial, or position. Morevoer, the type of question should be yes/no and open-end and the answer cannot be more than 3 word.
case 0:
Description 1: a man is standing in front of a large screen that is turned on he is wearing a blue shirt andblue headphones he is also wearing a purple shirt and a purple vest there is a large mirror on the wall behind the man.
Description 2: a person is standing and he is wearing headphones on his head he is holding a microphone in his handhe is also wearing a blue tshirt and i can also see a microphone on the right side of the image there is a wall.
case 0:
Q:How many people are there? A:1 Topic:

In [70]:
def chatgpt(sub_df, idx, requirement, start_time):
    caption_1 = list(sub_df["caption_1"])
    caption_2 = list(sub_df["caption_2"])
    prompt = requirement
    for i, (c1,c2) in enumerate(zip(caption_1, caption_2)):
        prompt += f"\nCase {i+1}\n"
        prompt += f"Description 1: {c1}\n"
        prompt += f"Description 2: {c2}\n"
    
    return prompt 

In [71]:

def qa_generator(csv_path, requirement, demo = True):
    start_time = time.time()
    df = pd.read_csv(csv_path)
    if demo:
        df = df[:9]
    df = df[["id", "img", "caption_1", "caption_2"]]
    df = df.rename(columns={'id': 'img_id'})
    df['id'] = df.index
    df["caption_1"] = df["caption_1"].map(caption_processing)
    df["caption_2"] = df["caption_2"].map(caption_processing)
    df["img"] = df["img"].map(get_img_pth)
    result_cases = []
    for idx in range(int(len(df)/3)):
        sub_df = df[idx*3:(idx+1)*3]
        prompt = chatgpt(sub_df, idx, requirement, start_time)


    return prompt

result = qa_generator("clean_cartoon.csv", requirement)

In [73]:
print(result)

You are given three cases with two different descriptions of an image each. You are required to ask 10 questions, their topics, and their answers based on the two descriptions for each case. The questions must be suitable for preschool students and should be about counting, reasoning, object detection, action recognition, attribute classification, comparative, spatial, or position. Morevoer, the type of question should be yes/no and open-end and the answer cannot be more than 3 word.
case 0:
Description 1: a man is standing in front of a large screen that is turned on he is wearing a blue shirt andblue headphones he is also wearing a purple shirt and a purple vest there is a large mirror on the wall behind the man.
Description 2: a person is standing and he is wearing headphones on his head he is holding a microphone in his handhe is also wearing a blue tshirt and i can also see a microphone on the right side of the image there is a wall.
case 0:
Q:How many people are there? A:1 Topic:

In [74]:
result = get_questions_anwers(result)

In [75]:
print(result)

Case 1:
Q:How many people are there? A:Unknown Topic:Counting.
Q:What are they playing? A:Game Topic:Object detection.
Q:Are there any pictures? A:Yes Topic:Object detection.
Q:What color is their skin? A:Yellow Topic:Attribute classification.
Q:Is there a musical instrument? A:Unknown Topic:Object detection.
Q:What is on the table? A:Game Topic:Object detection.
Q:How many frames on the wall? A:Unknown Topic:Counting.
Q:What is the background color? A:Unknown Topic:Attribute classification.
Q:What is the location of the frames? A:Wall Topic:Position.
Q:What is the game about? A:Unknown Topic:Reasoning.

Case 2:
Q:How many people? A:Multiple Topic:Counting.
Q:Are they sitting or standing? A:Sitting/standing Topic:Action recognition.
Q:What is in the background? A:Door Topic:Object detection.
Q:What color is their skin? A:Yellow Topic:Attribute classification.
Q:What are they listening to? A:Speech Topic:Object detection.
Q:What is the audience doing? A:Sitting Topic:Action recognition.

In [76]:
def extract_cases(string):
    string = string.replace("For case", "Case")
    string = string.replace("For Case", "Case")
    string = string.replace("\n\n", "\n")
    
    cases = []
    
    # Split the string by 'Case' to separate each case
    case_strings = string.split('Case ')[1:]
    for case_string in case_strings:
        case = {}
        case_string= case_string[2:]
        print(case_string)
        # Split the case string by newline character '\n' to separate each question-answer pair
        pairs = case_string.split('\nQ:')[1:]
        print(pairs)
        
        case['questions'] = []
        case['answers'] = []
        case['topics'] = []
        
        for pair in pairs:
            # Split the pair by 'Q:', 'A:', and 'Topic:' to extract the question, answer, and topic respectively
            question, answer, topic = pair.split(' A:', 1)[0].strip(), pair.split(' A:', 1)[1].split(' Topic:', 1)[0].strip(), pair.split(' Topic:', 1)[1].strip()
            
            case['questions'].append(question)
            case['answers'].append(answer)
            case['topics'].append(topic)
        
        cases.append(case)
    return cases
extract_cases(result)


Q:How many people are there? A:Unknown Topic:Counting.
Q:What are they playing? A:Game Topic:Object detection.
Q:Are there any pictures? A:Yes Topic:Object detection.
Q:What color is their skin? A:Yellow Topic:Attribute classification.
Q:Is there a musical instrument? A:Unknown Topic:Object detection.
Q:What is on the table? A:Game Topic:Object detection.
Q:How many frames on the wall? A:Unknown Topic:Counting.
Q:What is the background color? A:Unknown Topic:Attribute classification.
Q:What is the location of the frames? A:Wall Topic:Position.
Q:What is the game about? A:Unknown Topic:Reasoning.

['How many people are there? A:Unknown Topic:Counting.', 'What are they playing? A:Game Topic:Object detection.', 'Are there any pictures? A:Yes Topic:Object detection.', 'What color is their skin? A:Yellow Topic:Attribute classification.', 'Is there a musical instrument? A:Unknown Topic:Object detection.', 'What is on the table? A:Game Topic:Object detection.', 'How many frames on the wall? 

[{'questions': ['How many people are there?',
   'What are they playing?',
   'Are there any pictures?',
   'What color is their skin?',
   'Is there a musical instrument?',
   'What is on the table?',
   'How many frames on the wall?',
   'What is the background color?',
   'What is the location of the frames?',
   'What is the game about?'],
  'answers': ['Unknown',
   'Game',
   'Yes',
   'Yellow',
   'Unknown',
   'Game',
   'Unknown',
   'Unknown',
   'Wall',
   'Unknown'],
  'topics': ['Counting.',
   'Object detection.',
   'Object detection.',
   'Attribute classification.',
   'Object detection.',
   'Object detection.',
   'Counting.',
   'Attribute classification.',
   'Position.',
   'Reasoning.']},
 {'questions': ['How many people?',
   'Are they sitting or standing?',
   'What is in the background?',
   'What color is their skin?',
   'What are they listening to?',
   'What is the audience doing?',
   'What is the event they are attending?',
   'Where is the location of t